Load pre-trained DiffuGPT Model.  

The pipeline is copied from `inf_diffugpt.py` from the DiffuGPT paper. They load the weights from the hub using `strict=False`, and ingore the missin `LMHead`. Effectively loading the weights from DiffuGPT and the LMHead from gpt2. Makes sense? Apparently...

In [71]:
from safetensors.torch import load_file
import model
from importlib import reload

reload(model)

# Load model
model = model.GPT2("local_config.json")

# Get safetensors
safetensor_file_path = "/mnt/c/users/giaco/Downloads/model.safetensors"
model_weights = load_file(safetensor_file_path)

# Adjust names
state_dict = {k.replace("denoise_model", "gpt2.transformer"):v for k,v in model_weights.items()}
state_dict["gpt2.transformer.wte.weight"] = state_dict["embed_tokens.weight"].clone()
del state_dict["embed_tokens.weight"]

# Load weights to the model
model.load_state_dict(state_dict, strict=False)

Modifying HF GPT2Model to accept custom attention mask...
Loading pre-trained gpt2...


_IncompatibleKeys(missing_keys=['gpt2.lm_head.weight'], unexpected_keys=[])

Generate with pre-trained model.

![dump/diffugpt_tokenizer.png](dump/diffugpt_tokenizer.png)

In [158]:
import tiktoken
import torch
from utils import get_annealing_mask

prompt = "Oh mama, what is your name"

tokenizer = tiktoken.get_encoding("gpt2")

input_ids = torch.tensor(
    tokenizer.encode(prompt)
).unsqueeze(0)

msk = get_annealing_mask(input_ids.shape[1], input_ids.shape[0], 1.0)

output_ids = model.generate(input_ids, max_new_tokens=10)

out = tokenizer.decode(
    output_ids[0].tolist()
)

# Logging
print("prompt: ", prompt)
print("input_ids:", input_ids.shape)
print("logits: ", output_ids.shape)
print("out: ", out)

prompt:  Oh mama, what is your name
input_ids: torch.Size([1, 8])
logits:  torch.Size([1, 18])
out:  Oh mama, what is your name, ingredients name choice what your your, is what
